## create single json file from multy json files 
- 
- merge multiple json files into single json file
- create bbox from segmentation labels 
- reindexing annotataions's and images's id 
- is_crowd -> iscrowd (error solved)

In [4]:
import datetime
import json
import os
import time 

tik = time.time()
data_path = './custom_object_segmentation/guardrail_json_120922/val1/'
new_json_path = './custom_object_segmentation/guardrail_json_120922/'
file_format = '.json'
new_json_file = 'val5'

class JsonAnt:
    
        # LOAD DATA
    def __init__(self):
        self.createDict()
        
    # reindex files
    def reindex(self, jsonAnt):
        # reindex images
        for imgId, bi in enumerate(jsonAnt['images']):
            bi['id'] += 24802 # for the trainng set it to 1 and for validation change it to 1+number of images proccssed in training part
        # reindex annotations  
        for annId, b in enumerate(jsonAnt['annotations']):
            b['id'] = annId+915389  # for the trainng set it to 1 and for validation change it to 1+number of annnnotations proccssed in training part
            b['iscrowd'] = b.pop('is_crowd')
        return jsonAnt
    
    def createDict(self):
        print('Creating dict')
        jsonAnt = {"info": [        {
            "contributor": "Konkuk_university",
            "date_created": "2021/05/21",
            "description": "PGON JSON file",
            "url": "",
            "version": "1.0",
            "year": 2021
        }],
           "images": [],
           "annotations": [],
           "categories": [
            {
                "id": 1,
                "name": "Building, Overpass, Bridge"
            },
            {
                "id": 2,
                "name": "Road Marking, Lane Marking"
            },
            {
                "id": 3,
                "name": "Guard Rail, Fence, Soundproof Wall"
            }]
          }
        return jsonAnt
    def write_json(self, jsonAnt, loaded_json, num_found_files):
        source_infos = loaded_json['info']
        source_images = loaded_json['images']
        source_images['id'] = num_found_files 
        source_categories = loaded_json['categories']
        source_annotations = loaded_json['annotations']        
        for _ in source_annotations:
            seg_list_b = _['segmentation'][0]
            xcoords, ycoords = [], []
            for i in range(0, len(seg_list_b), 2):
                xcoords.append(seg_list_b[i])
            for i in range(1, len(seg_list_b), 2):
                ycoords.append(seg_list_b[i])
            xmax, ymin = float(max(xcoords)), float(min(ycoords))
            ymax, xmin = float(max(ycoords)), float(min(xcoords))
            poly_bbox = [xmax, ymin, ymax, xmin]
            width, height = xmax - xmin, ymax - ymin
            bboxx = [int(xmin), int(ymin), int(width), int(height)]
            _['bbox'] = bboxx
            _['image_id'] = num_found_files+24802 # for the trainng set it to 1 and for validation change it to 1+number of images proccssed in training part
        jsonAnt['images'].append(source_images)
        jsonAnt['annotations'] += source_annotations
        return jsonAnt


# RUN THE PROGRAM
def main():
    
    callClass = JsonAnt()
    
    json_Ant  = callClass.createDict()
    num_found_files = 0
    print(' Loading Coco json')
    for file in os.listdir(data_path):
        try:
            if 'json' in file:
                num_found_files = num_found_files + 1
                json_path = os.path.join(data_path, file)
                with open(json_path) as f:
                    loaded_json = json.load(f)
                callClass.write_json(json_Ant, loaded_json, num_found_files)
        except:
            if 'json' not in file:
                print(' Finished, nothing to load')
                return

    callClass.reindex(json_Ant)

    now = datetime.datetime.now()        # creating new file name with date attached
    currentDate = "_" + str(now.day) + "_" + str(now.hour) + "_" + str(now.minute) + "_" + str(now.second)
    file_output = os.path.join(new_json_file + file_format)
     
    with open(new_json_path + file_output, 'a+') as outfile:
        json.dump(json_Ant, outfile, sort_keys=True, indent=4, )  # , indent=4, im.__dict__
        outfile.write('\n')
        outfile.close()
    print(f'    Location:  {new_json_path}  \n Output file:  {file_output} \n')
    
    
if __name__ == '__main__':
    main()
    
tok = time.time()

print(tok-tik)

Creating dict
Creating dict
 Loading Coco json
    Location:  ./custom_object_segmentation/guardrail_json_120922/  
 Output file:  val5.json 

16.50649857521057
